# 安裝MSSQLdb

In [9]:
! pip install pyodbc

# MSSQL 連線與寫入DB

In [36]:
import pyodbc


def __init__(self, table_name):
        connection_string = 'DRIVER={SQL Server Native Client 11.0};SERVER=(LocalDB)\V11.0;DATABASE=TMSDB;Persist security info=TRUE;Integrated Security=TRUE;Trusted_Connection=Yes; MultipleActiveResultSets = true'
        self.conn = pypyodbc.connect(connection_string)
        self.c = self.conn.cursor()
        self.table_name = table_name
#         print(self.table_name)

def insert(self,input):
        # Insert a row of data
        temp_str = 'INSERT INTO '+self.table_name+' VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)'        
        self.c = self.conn.cursor()
        self.c.execute(temp_str, input)

def update(self,temp_str):
        self.cursor().execute(temp_str)
        self.commit()
        
def commit(self):
        #self.c = self.cursor()
        # Save (commit) the changes
        self.cursor().commit()   
        
def close(self):
        self.c = self.conn.cursor()
        self.c.close()
        self.conn.close()       
        

## 連線到(LocalDB)\V11.0

In [138]:
import pyodbc

results=[]
def getADDRList():
    with pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=(LocalDB)\V11.0;DATABASE=TMSDB; \
                          Persist security info=TRUE;Integrated Security=TRUE;Trusted_Connection=Yes; \
                          MultipleActiveResultSets = true') as cnxn:
        cursor = cnxn.cursor()
        cursor.execute("SELECT TOP (10000) [ADDR_ID] \
              ,[ADDRESS] \
              ,[LAT] \
              ,[LONG] \
          FROM [TMSDB].[dbo].[addrlist] \
          WHERE LAT IS NULL OR LONG IS NULL")

        columns= [column[0] for column in cursor.description]
        
        for row in cursor.fetchall():
            print(type(row))
            results.append(dict(zip(columns,row)))
            print(results)
            break
        return results
getADDRList()

<class 'pyodbc.Row'>
[{'ADDR_ID': 23925561, 'ADDRESS': '242新北市新莊區豐年街１２４巷６弄４號五樓', 'LAT': None, 'LONG': None}]


[{'ADDRESS': '242新北市新莊區豐年街１２４巷６弄４號五樓',
  'ADDR_ID': 23925561,
  'LAT': None,
  'LONG': None}]

## 更新至addrlist

In [ ]:
def updateaddrdb():
    with pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=(LocalDB)\V11.0;DATABASE=TMSDB; \
        Persist security info=TRUE;Integrated Security=TRUE;Trusted_Connection=Yes; \
        MultipleActiveResultSets = true') as cnxn:
    updatestr='UPDATE DBO.addrlist SET LAT={},LONG={} WHERE ADDR_ID={}'.format(lat,long,addrID)
    #print(updatestr)
    update(cnxn,updatestr)
    commit(cnxn)

In [111]:
convertedary=[]
with open('sucaddrid.csv' , 'r') as f:
      for l in f:
        convertedary.append(l.split(',')[0])
if '23925561' in convertedary:
    print(str(addrID) + 'ssss\n')
else:
    print('pass')

23925561ssss



### 連線MOTC轉地址
     INPUT:DB
     OUTPUT:sucaddrid.txt
     ERRFILE:erraddrid.txt

In [49]:
import pyodbc
import requests
import time
from urllib.parse import quote
import pandas

url='https://gist.motc.gov.tw/gist_api/V2/Map/GeoCode/Address/'
url2='?$format=GEOJSON'
results=[]

convertedary=[]
with open('sucaddrid.csv' , 'r') as f:
      for l in f:
        convertedary.append(l.split(',')[0])
        #print(l.split(',')[0])
#print(convertedary)
#return
#連線到SQL SERVER抓未轉換地址
#result = getADDRList()
#連線到SQL SERVER抓未轉換地址 end        
#開檔案抓需轉換地址
df = pandas.read_csv('addrlist3.csv',encoding = 'utf8')
for idx,ele in df.iterrows():
    dictitem = {'ADDR_ID' : ele[0],'ADDRESS' : ele[1]}
    results.append(dictitem)

#開檔案抓需轉換地址 end
for r in results:
    addrID=r.get('ADDR_ID')
    addr=r.get('ADDRESS')
    addr=addr.replace("(住)", "").strip()

    #過濾已轉換的地址代碼
    if str(addrID) in convertedary:
        #print(str(addrID) + 'ssss\n')
        continue
    print(addrID)
    try:
        #addr = '台北市內湖區瑞光路414號'
        if (addr != None) :
            addurl = '{}{}{}'.format(url,addr,url2)
            #print(addurl)
            headers = {
             'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.168 Safari/537.36'
            }
            res = requests.get(quote(addurl, safe='/:?$='), headers = headers, verify = False)
            jd = res.json()
            #print(jd)
            addcorr = pandas.DataFrame(jd['features'][0]['geometry'])
            #print(addcorr)
            long=addcorr['coordinates'][0]
            lat=addcorr['coordinates'][1]
            #print(addr,lat,long)

            with open('sucaddrid.csv' , 'a') as f:
                f.write(str(addrID) + ',' + addr + ',' + str(lat) + ',' + str(long) +'\n')
            #print('success')
            time.sleep(0.5)
    except Exception as e:
        print('fail:' + str(e))
        with open('erraddrid.csv' , 'a') as f:
            f.write(str(addrID) + '\n')
        time.sleep(5)

24135746


C:\Program Files\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


24142473


C:\Program Files\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


24142566


...

23859876


C:\Program Files\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


23859943
fail:HTTPSConnectionPool(host='gist.motc.gov.tw', port=443): Max retries exceeded with url: /gist_api/V2/Map/GeoCode/Address/235%E6%96%B0%E5%8C%97%E5%B8%82%E4%B8%AD%E5%92%8C%E5%8D%80%E4%BF%9D%E5%81%A5%E8%B7%AF%EF%BC%94%EF%BC%96%E5%B7%B7%EF%BC%92%EF%BC%96%E5%BC%84%EF%BC%96%E8%99%9F?$format=GEOJSON (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000000009CF434A8>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。',))
23859961


C:\Program Files\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


KeyboardInterrupt: 

### 用GOOGLEGEO CODING API轉地址

In [ ]:
import urllib, json, time

def GoogGeoAPI(address,api="",delay=5):
      base = r"https://maps.googleapis.com/maps/api/geocode/json?"
      addP = "address=" + address.replace(" ","+")
      GeoUrl = base + addP + "&key=" + api
      response = requests.get(GeoUrl,verify = False)
      jsonData = response.json()
      if jsonData['status'] == 'OK':
        resu = jsonData['results'][0]
        finList = [resu['formatted_address'],resu['geometry']['location']['lat'],resu['geometry']['location']['lng']]
      else:
        finList = [None,None,None]
      time.sleep(delay) #in seconds
      return finList